### Install below libraries in order to run the rest of the notebook
documentation: https://open-meteo.com/en/docs
- !pip install openmeteo-requests
- !pip install requests-cache retry-requests numpy pandas

In [86]:
# Import dependencies

import json
import openmeteo_requests # need to install this library with above cell
import requests_cache # need to install this library with above cell
from retry_requests import retry # need to install this library with above cell
import pandas as pd
import datetime as dt


In [71]:
# Date range
start_date = '2013-01-01'
end_date = '2023-12-31'

# Coordinates - focusing only on Manhattan
central_park_lat = 40.7896239
central_park_lon = -73.9598939
wash_heights_lat = 40.8417052861612
wash_heights_lon = -73.93934920585686
west_vill_lat = 40.73473345314149
west_vill_lon = -74.00460403083804
midtown_lat = 40.75504999601737
midtown_lon = -73.98400001962062
fin_dist_lat = 40.707717754058876
fin_dist_lon = -74.00783736342065


# bushwick_lat = 40.69481172967752
# bushwick_lon = -73.91353551303266
# rockaway_lat = 40.58793733907492
# rockaway_lon = -73.80887880370675



In [72]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [73]:
# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": [central_park_lat, wash_heights_lat, west_vill_lat, midtown_lat, fin_dist_lat],
	"longitude": [central_park_lon, wash_heights_lon, west_vill_lon, midtown_lon, fin_dist_lon],
	"start_date": start_date,
	"end_date": end_date,
	"daily": ["temperature_2m_max", "temperature_2m_min", "temperature_2m_mean"],
	"temperature_unit": "fahrenheit",
	"timezone": "America/New_York"
}
responses = openmeteo.weather_api(url, params=params)

In [74]:
# Process data. Set empty lists and define neighborhoods variable.
neighborhoods = ['Central Park', 'Washington Heights','West Village','Midtown', 'Financial District']
min_temp = []
max_temp = []
mean_temp = []
location = []

# Loop through responses, define lists from each response, assign element of neighborhood to name variable
for x in range(len(responses)):
    response = responses[x]
    daily = response.Daily()
    daily_temperature_2m_max = list(daily.Variables(0).ValuesAsNumpy())
    daily_temperature_2m_min = list(daily.Variables(1).ValuesAsNumpy())
    daily_temperature_2m_mean = list(daily.Variables(2).ValuesAsNumpy())
    name = neighborhoods[x]
    
    # Loop through number of elements in daily_temperature lists (4017), append each from each list into location, min_temp, max_temp, mean_temp lists
    for y in range(len(daily_temperature_2m_max)):
        location.append(name)
        min_temp.append(daily_temperature_2m_min[y])
        max_temp.append(daily_temperature_2m_max[y])
        mean_temp.append(daily_temperature_2m_mean[y])

In [140]:
# Define the first element in responses to response, then get use .Daily() to get date value
response = responses[0]
daily = response.Daily()

# Create DatetimeIndex object using pd.date_range into date_set
date_set = pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
    )

# Create copy of date_set into date
date = date_set

# Append date_set values into date 4 more times
for x in range(4):
    date = date.append(date_set)

In [146]:
# Create DataFrame
temp_df = pd.DataFrame({
    "Date": date,
    "Location" : location,
    "Min Temp": min_temp,
    "Max Temp": max_temp,
    "Mean Temp": mean_temp
    })

In [157]:
# Display info
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20085 entries, 0 to 20084
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype              
---  ------     --------------  -----              
 0   Date       20085 non-null  datetime64[ns, UTC]
 1   Location   20085 non-null  object             
 2   Min Temp   20085 non-null  float32            
 3   Max Temp   20085 non-null  float32            
 4   Mean Temp  20085 non-null  float32            
dtypes: datetime64[ns, UTC](1), float32(3), object(1)
memory usage: 549.3+ KB


In [159]:
# Preview DataFrame
temp_df.head(20)

,Date,Location,Min Temp,Max Temp,Mean Temp
0,2013-01-01 05:00:00+00:00,Central Park,23.239399,37.279400,32.835651
1,2013-01-02 05:00:00+00:00,Central Park,18.289400,30.169399,23.445648
2,2013-01-03 05:00:00+00:00,Central Park,15.139400,30.529400,22.826899
3,2013-01-04 05:00:00+00:00,Central Park,23.959400,35.749401,29.618151
4,2013-01-05 05:00:00+00:00,Central Park,28.729401,39.079399,32.089397
5,2013-01-06 05:00:00+00:00,Central Park,26.029400,41.419399,32.246899
6,2013-01-07 05:00:00+00:00,Central Park,23.869400,42.589401,32.104397
7,2013-01-08 05:00:00+00:00,Central Park,26.569401,46.009399,34.271900
8,2013-01-09 05:00:00+00:00,Central Park,29.089399,46.369400,38.813152
9,2013-01-10 05:00:00+00:00,Central Park,28.999401,46.009399,37.774403


In [161]:
# Save temp_df to CSV file
temp_df.to_csv('Output/Manhattan_Temperatures_2013-2023.csv', index=False)